# Read the records from the Simple Test of Reciprocity

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from sys import argv
from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize


from pyaspect.project import *
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import *
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import *
from pyaspect.specfemio.write import *
from pyaspect.specfemio.write import _write_header
from pyaspect.specfemio.read import _read_headers
from pyaspect.specfemio.read import *
from pyaspect.specfemio.utils import *


import pyaspect.events.gevents as gevents
import pyaspect.events.gstations as gstations
from pyaspect.events.munge.knmi import correct_station_depths as csd_f
import pyaspect.events.mtensors as mtensors
from obspy.imaging.beachball import beach
from obspy import UTCDateTime
import shapefile as sf

## Set Project Paths

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
projects_fqp = os.path.join(data_out_dir,'tmp','TestProjects','Computed_Forward_and_Reciprocity_Test')
recip_project_fqp = os.path.join(projects_fqp,'ReciprocalTestProject')
fwd_project_fqp = os.path.join(projects_fqp,'ForwardTestProject')
!ls {recip_project_fqp}
print()
!ls {fwd_project_fqp}

## Read Reciprocal Project RecordHeader and load data

In [ ]:
import glob
recip_record_fqp = os.path.join(recip_project_fqp,'pyheader.project_record')
recip_record_h = _read_headers(recip_record_fqp)

ne = recip_record_h.nevents
ns = recip_record_h.nsrc

'''
''';
l_data_x = []
l_data_y = []
l_data_z = []
for idx, row in recip_record_h.stations_df.iterrows():
    #fp_prefix = row['data_fqdn'].values[0]
    fp_prefix = row['data_fqdn']
    fp = os.path.join(projects_fqp,fp_prefix) 
    for comp in ['X','Y','Z']:
        match_fp = fp + '.*X' + comp + '.semd'
        cname = 'comp_' + comp
        for filepath in glob.glob(match_fp):
            #print(f'np.fromfile({filepath}, dtype=np.float32)')
            if comp == 'X':
                #l_data_x.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
                l_data_x.append(np.fromfile(filepath, dtype=np.float32))
            elif comp == 'Y':
                #l_data_y.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
                l_data_y.append(np.fromfile(filepath, dtype=np.float32))
            elif comp == 'Z':
                #l_data_z.append((fp_prefix,np.fromfile(filepath, dtype=np.float32)))
                l_data_z.append(np.fromfile(filepath, dtype=np.float32))
            else:
                assert False
            
recip_record_h.stations_df['comp_EX'] = l_data_x
recip_record_h.stations_df['comp_NY'] = l_data_y
recip_record_h.stations_df['comp_Z']  = l_data_z

'''
#print(f'Record w/ data:\n\n{recip_record_h.stations_df}')
for idx, row in recip_record_h.stations_df.iterrows():
    fp_prefix = row['data_fqdn']
    x_pre = row['comp_EX'][0]
    y_pre = row['comp_NY'][0]
    z_pre = row['comp_Z'][0]
    assert x_pre == fp_prefix
    assert y_pre == fp_prefix
    assert z_pre == fp_prefix
    #print(f'row["comp_EX"] = {row["comp_EX"]}')
''';
    
print(f'Record w/ data:\n\n{recip_record_h.stations_df["comp_Z"]}')

'''
for eidx, edf in recip_record_h.stations_df.groupby(level='eid'):
    assert ns == edf.index.get_level_values('sid').nunique()
    for sidx, sdf in edf.groupby(level='sid'):
        for ridx, rdf in sdf.groupby(level='trid'):
            for gidx, gdf in rdf.groupby(level='gid'):
                fp_prefix = gdf['data_fqdn'].values[0]
                fp = os.path.join(projects_fqp,fp_prefix) 
                for comp in ['X','Y','Z']:
                    match_fp = fp + '.*X' + comp + '.semd'
                    cname = 'comp_' + comp
                    for filepath in glob.glob(match_fp):
                        #print(f'np.fromfile({filepath}, dtype=np.float32)')
                        gdf[cname] = np.fromfile(filepath, dtype=np.float32)
        

print(recip_record_h)
''';
    
    
'''
l_traces = []
for fp in recip_record_h.stations_df['data_fqdn'].tolist():
    
    fp_prefix = os.path.join(projects_fqp,fp) 
    match_fp = fp_prefix + '.*XZ.semd'
    #print(f'fp_prefix: {fp_prefix}')
    #print(f'match_fp: {match_fp}')
    
    for filepath in glob.glob(match_fp):
        #print(f'files:\n{filepath}')
        zar = np.fromfile(filepath, dtype=np.float32)
        #print(f'zar.shape = {zar.shape}')
        l_traces.append(zar)

traces_z = np.array(l_traces)
print(f'traces_z.shape: {traces_z.shape}')
''';
    
      
'''
for idx, rec in recip_record_h.stations_df['data_fqdn'].iterrows():
    print(f'idx: {idx}')
    print(f'rec:\n{rec}')
''';

'''
plt.plot(traces_z[100])
plt.show()
''';

In [ ]:

x = np.array([np.array([1,2,3],dtype=np.float32),np.array([4,5,6,7],dtype=np.float32)],dtype=object)

In [ ]:
print(x[0][1]) 
print(x[1][0]) 

In [ ]:
type(x) 